+ [x] Edge Loss の調整：w_edge を調整して“線のシャープさ”と“ノイズ”のトレードオフを取ってください（5〜20の範囲で試すと良い）。

+ [ ] Perceptual Loss（VGG）を加えると線の太さや“見た目”が向上することがあります（ただし学習が複雑に）。torchvision.models.vgg19(pretrained=True) の中間特徴で L1 を取る方式。

+ [ ] PatchGAN 判別器（GAN Loss） を追加すると更にシャープで自然な線が出せます。安定化には History Buffer/LSGAN/PatchGAN が使われます。

+ [ ] Deep Supervision：複数解像度で出力し、それぞれに損失を付けると細線の維持に有効です。

+ [ ] Data augment：線の太さや明度変化、ランダムブラッシュストロークなどを入れると汎化が良くなります。

+ [ ] Inference TTA：左右反転やスケールを平均化して最終出力を滑らかにする。

In [2]:
# train.py
import math
import os
from typing import Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import glob

from torchviz import make_dot

2025-11-15 14:09:38.373656713 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


In [3]:
# -----------------------
# Utility blocks
# -----------------------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=3, stride=1, padding=1, use_bn=True):
        super().__init__()
        layers = [nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias=not use_bn)]
        if use_bn:
            layers.append(nn.InstanceNorm2d(out_ch, affine=True))
        layers.append(nn.ReLU(inplace=True))
        self.net = nn.Sequential(*layers)
    def forward(self, x): return self.net(x)

class ResidualBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.conv1 = ConvBlock(ch, ch)
        self.conv2 = nn.Conv2d(ch, ch, 3, 1, 1, bias=False)
        self.norm = nn.InstanceNorm2d(ch, affine=True)
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.norm(out)
        return F.relu(out + x)

# -----------------------
# Simple Attention Gate (spatial attention)
# -----------------------
class SpatialAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3)
    def forward(self, x):
        # channel-wise average and max
        avg = x.mean(dim=1, keepdim=True)
        mx, _ = x.max(dim=1, keepdim=True)
        cat = torch.cat([avg, mx], dim=1)  # 2 x H x W
        # reduce to single-channel attention
        att = torch.sigmoid(self.conv(cat))
        return x * att

# -----------------------
# Basic Self-Attention module (non-local)
# -----------------------
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.query_conv = nn.Conv2d(in_dim, in_dim//8, 1)
        self.key_conv   = nn.Conv2d(in_dim, in_dim//8, 1)
        self.value_conv = nn.Conv2d(in_dim, in_dim, 1)
        self.gamma = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        b, c, h, w = x.size()
        proj_query = self.query_conv(x).view(b, -1, h*w).permute(0,2,1)  # B x N x C'
        proj_key   = self.key_conv(x).view(b, -1, h*w)                    # B x C' x N
        energy = torch.bmm(proj_query, proj_key)                          # B x N x N
        att = F.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(b, -1, h*w)                  # B x C x N
        out = torch.bmm(proj_value, att.permute(0,2,1))                   # B x C x N
        out = out.view(b, c, h, w)
        out = self.gamma * out + x
        return out

# -----------------------
# U-Net with Residual and Attention
# -----------------------
class AttnResUNet(nn.Module):
    def __init__(self, in_ch=1, out_ch=1, base_ch=64, use_self_attn=True):
        super().__init__()
        self.enc1 = nn.Sequential(ConvBlock(in_ch, base_ch), ResidualBlock(base_ch))
        self.pool1 = nn.MaxPool2d(2)

        self.enc2 = nn.Sequential(ConvBlock(base_ch, base_ch*2), ResidualBlock(base_ch*2))
        self.pool2 = nn.MaxPool2d(2)

        self.enc3 = nn.Sequential(ConvBlock(base_ch*2, base_ch*4), ResidualBlock(base_ch*4))
        self.pool3 = nn.MaxPool2d(2)

        self.bottleneck = nn.Sequential(ConvBlock(base_ch*4, base_ch*8), ResidualBlock(base_ch*8))
        self.self_attn = SelfAttention(base_ch*8) if use_self_attn else nn.Identity()

        # decoder
        self.up3 = nn.ConvTranspose2d(base_ch*8, base_ch*4, 2, 2)
        self.dec3 = nn.Sequential(ConvBlock(base_ch*8, base_ch*4), ResidualBlock(base_ch*4))

        self.up2 = nn.ConvTranspose2d(base_ch*4, base_ch*2, 2, 2)
        self.dec2 = nn.Sequential(ConvBlock(base_ch*4, base_ch*2), ResidualBlock(base_ch*2))

        self.up1 = nn.ConvTranspose2d(base_ch*2, base_ch, 2, 2)
        self.dec1 = nn.Sequential(ConvBlock(base_ch*2, base_ch), ResidualBlock(base_ch))

        self.final = nn.Conv2d(base_ch, out_ch, 1)
        self.spatial_attn1 = SpatialAttention()
        self.spatial_attn2 = SpatialAttention()
        self.spatial_attn3 = SpatialAttention()

    def forward(self, x):
        e1 = self.enc1(x)           # B,64,H,W
        p1 = self.pool1(e1)

        e2 = self.enc2(p1)          # B,128,H/2,W/2
        p2 = self.pool2(e2)

        e3 = self.enc3(p2)          # B,256,H/4,W/4
        p3 = self.pool3(e3)

        b = self.bottleneck(p3)     # B,512,H/8,W/8
        b = self.self_attn(b)

        u3 = self.up3(b)            # B,256,H/4,W/4
        # attention on skip
        e3_att = self.spatial_attn1(e3)
        d3 = self.dec3(torch.cat([u3, e3_att], dim=1))

        u2 = self.up2(d3)
        e2_att = self.spatial_attn2(e2)
        d2 = self.dec2(torch.cat([u2, e2_att], dim=1))

        u1 = self.up1(d2)
        e1_att = self.spatial_attn3(e1)
        d1 = self.dec1(torch.cat([u1, e1_att], dim=1))

        out = self.final(d1)
        out = torch.sigmoid(out)  # output in [0,1]
        return out

# -----------------------
# Sobel (Edge) loss
# -----------------------
class SobelEdgeLoss(nn.Module):
    def __init__(self, p=1):
        super().__init__()
        # Sobel kernels (horizontal, vertical)
        kx = torch.tensor([[1, 0, -1],
                           [2, 0, -2],
                           [1, 0, -1]], dtype=torch.float32) / 4.0
        ky = torch.tensor([[1, 2, 1],
                           [0, 0, 0],
                           [-1, -2, -1]], dtype=torch.float32) / 4.0
        self.register_buffer('kx', kx.view(1,1,3,3))
        self.register_buffer('ky', ky.view(1,1,3,3))
        self.p = p

    def forward(self, pred, target):
        # pred/target: Bx1xHxW
        kx = self.kx.to(device=pred.device, dtype=pred.dtype)
        ky = self.ky.to(device=pred.device, dtype=pred.dtype)

        gx_pred = F.conv2d(pred, kx, padding=1)
        gy_pred = F.conv2d(pred, ky, padding=1)
        gx_tgt  = F.conv2d(target, kx, padding=1)
        gy_tgt  = F.conv2d(target, ky, padding=1)

        edge_pred = torch.sqrt(gx_pred**2 + gy_pred**2 + 1e-6)
        edge_tgt  = torch.sqrt(gx_tgt**2 + gy_tgt**2 + 1e-6)

        if self.p == 1:
            return F.l1_loss(edge_pred, edge_tgt)
        else:
            return F.mse_loss(edge_pred, edge_tgt)


# -----------------------
# Small demo dataset (grayscale pairs)
# Replace with your pair dataset: (rough_sketch, line_art)
# -----------------------
class PairImageDataset(Dataset):
    def __init__(self, rough_dir, line_dir, size=(256,256)):
        super().__init__()
        self.rough_files = sorted(glob.glob(os.path.join(rough_dir, '*.jpg')))
        self.line_files  = sorted(glob.glob(os.path.join(line_dir, '*.jpg')))
        assert len(self.rough_files) == len(self.line_files), "pair counts must match"
        self.tr = transforms.Compose([
            transforms.Resize(size),
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor()
        ])
    def __len__(self): return len(self.rough_files)
    def __getitem__(self, idx):
        r = Image.open(self.rough_files[idx]).convert('RGB')
        l = Image.open(self.line_files[idx]).convert('RGB')
        return self.tr(r), self.tr(l)

# -----------------------
# Training loop
# -----------------------
def train(
    rough_dir='dataset/train/rough', line_dir='dataset/train/line',
    epochs=50, batch_size=8, lr=1e-4, device='cuda'
):
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    dataset = PairImageDataset(rough_dir, line_dir)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    model = AttnResUNet(in_ch=1, out_ch=1, base_ch=64, use_self_attn=True).to(device)
    edge_loss = SobelEdgeLoss()
    l1_loss = nn.L1Loss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.5, 0.999), weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs*len(loader))

    scaler = torch.cuda.amp.GradScaler()  # mixed precision
    save_dir = 'checkpoints'
    os.makedirs(save_dir, exist_ok=True)

    # loss weights
    w_l1 = 1.0
    w_edge = 10.0   # emphasize edges

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0
        for i, (rough, line) in enumerate(loader):
            rough = rough.to(device)
            line = line.to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                out = model(rough)
                loss_l1 = l1_loss(out, line)
                loss_edge = edge_loss(out, line)
                loss = w_l1 * loss_l1 + w_edge * loss_edge

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            running_loss += loss.item()
            if (i+1) % 50 == 0:
                print(f"Epoch[{epoch}/{epochs}] Iter[{i+1}/{len(loader)}] Loss: {running_loss / (i+1):.4f}")

        avg = running_loss / len(loader)
        print(f"==> Epoch {epoch} finished. avg loss: {avg:.4f}")

        # save checkpoint
        torch.save({
            'epoch': epoch,
            'model_state': model.state_dict(),
            'optim_state': optimizer.state_dict()
        }, os.path.join(save_dir, f'model_epoch_{epoch}.pth'))

    print("Training finished.")




In [4]:
if __name__ == "__main__":
    # Example: point to your directories containing paired PNGs
    train(
        rough_dir='dataset/train/rough',   # 下絵画像フォルダ（png）
        line_dir='dataset/train/line',     # 正解線画フォルダ（png）
        epochs=30,
        batch_size=8,
        lr=2e-4,
        device='cuda'
    )

==> Epoch 1 finished. avg loss: 0.9017
==> Epoch 2 finished. avg loss: 0.7092
==> Epoch 3 finished. avg loss: 0.6463
==> Epoch 4 finished. avg loss: 0.6163
==> Epoch 5 finished. avg loss: 0.5990
==> Epoch 6 finished. avg loss: 0.5880
==> Epoch 7 finished. avg loss: 0.5804
==> Epoch 8 finished. avg loss: 0.5737
==> Epoch 9 finished. avg loss: 0.5687
==> Epoch 10 finished. avg loss: 0.5645
==> Epoch 11 finished. avg loss: 0.5611
==> Epoch 12 finished. avg loss: 0.5586
==> Epoch 13 finished. avg loss: 0.5554
==> Epoch 14 finished. avg loss: 0.5540
==> Epoch 15 finished. avg loss: 0.5514
==> Epoch 16 finished. avg loss: 0.5491
==> Epoch 17 finished. avg loss: 0.5450
==> Epoch 18 finished. avg loss: 0.5417
==> Epoch 19 finished. avg loss: 0.5370
==> Epoch 20 finished. avg loss: 0.5299
==> Epoch 21 finished. avg loss: 0.5224
==> Epoch 22 finished. avg loss: 0.5123
==> Epoch 23 finished. avg loss: 0.5016
==> Epoch 24 finished. avg loss: 0.4891
==> Epoch 25 finished. avg loss: 0.4769
==> Epoch

In [5]:
# inference
def load_model(checkpoint_path, device='cuda'):
    model = AttnResUNet(in_ch=1, out_ch=1, base_ch=64, use_self_attn=True)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state'])
    model.to(device)
    model.eval()
    print(f"✅ Loaded checkpoint from {checkpoint_path}")
    return model

def preprocess_image(img_path, size=(256,256)):
    tr = transforms.Compose([
        transforms.Resize(size),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
    ])
    img = Image.open(img_path).convert('RGB')
    return tr(img).unsqueeze(0)  # 1x1xHxW tensor

def postprocess_and_save(tensor, save_path):
    # tensor: 1x1xHxW in [0,1]
    img = tensor.squeeze(0).squeeze(0).detach().cpu()  # HxW
    img = transforms.ToPILImage()(img)
    img.save(save_path)
    print(f"💾 Saved output to {save_path}")

@torch.no_grad()
def inference(model, input_path, output_path, device='cuda'):
    img = preprocess_image(input_path)
    img = img.to(device)

    with torch.amp.autocast(device_type='cuda'):
        output = model(img)

    postprocess_and_save(output, output_path)



In [6]:
if __name__ == "__main__":
    # ======== 設定 =========
    checkpoint_path = "checkpoints/model_epoch_30.pth"  # 学習済みモデル
    input_dir = "test/rough"   # テスト用の下絵
    output_dir = "results"             # 出力先
    os.makedirs(output_dir, exist_ok=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = load_model(checkpoint_path, device)

    # ======== 推論 =========
    rough_files = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    for fname in rough_files:
        in_path = os.path.join(input_dir, fname)
        out_path = os.path.join(output_dir, fname)
        inference(model, in_path, out_path, device)


✅ Loaded checkpoint from checkpoints/model_epoch_30.pth
💾 Saved output to results/sample.jpg
💾 Saved output to results/sample01.jpg


| パラメータ    | 効果                     | 備考                     |
| -------- | ---------------------- | ---------------------- |
| `w_l1`   | 全体の明るさや形の再現重視          | 通常 1.0 でOK             |
| `w_edge` | エッジ（線のシャープさ）をどれだけ重視するか | 大きいほど線がくっきり。ただしノイズも増える |


| 例      | 設定               | 効果の傾向                 |
| ------ | ---------------- | --------------------- |
| 柔らかめ   | `w_edge = 3.0`   | 線がややぼやけるがノイズ少ない       |
| 標準     | `w_edge = 5.0`   | バランス型（今の設定）           |
| シャープ強調 | `w_edge = 10.0`  | 線がくっきり、ややノイズ出る        |
| 強め     | `w_edge = 15〜20` | エッジ最重視、線は濃いがノイズが増える傾向 |


In [8]:
!ls -la results

total 48
drwxrwxr-x  2 sh1 sh1  4096 11月 15 14:04 .
drwxrwxr-x 13 sh1 sh1  4096 11月 15 14:04 ..
-rw-rw-r--  1 sh1 sh1 11303 11月 13 19:07 result_line.png
-rw-rw-r--  1 sh1 sh1  3360 11月 15 14:04 sample01.jpg
-rw-rw-r--  1 sh1 sh1  7018 11月 14 02:18 sample_w_edge10.jpg
-rw-rw-r--  1 sh1 sh1  5132 11月 14 02:01 sample_w_edge20.jpg
-rw-rw-r--  1 sh1 sh1  5574 11月 15 14:04 sample_w_edge8.jpg


result
![result](./results/sample_w_edge8.jpg) 8
![result](./results/sample_w_edge10.jpg) 10
![result_w_edge20](./results/sample_w_edge20.jpg) 20


sample  
![sample](./test/rough/sample.jpg)

In [7]:
import onnx
def export_to_onnx(model, input_size, onnx_path="model.onnx"):
    # 1. CPUへ移動
    model = model.to("cpu")
    model.eval()

    # 2. 入力もCPU
    x = torch.randn(*input_size, device="cpu")

    torch.onnx.export(
        model, x, onnx_path,
        opset_version=13,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output']
    )
    print(f"[OK] Exported ONNX -> {onnx_path}")



In [8]:
model = load_model("checkpoints/model_epoch_20.pth")
export_to_onnx(model, (1,1,256,256))

✅ Loaded checkpoint from checkpoints/model_epoch_20.pth
[OK] Exported ONNX -> model.onnx


/home/sh1/deepl/mynovel/venv/lib/python3.10/site-packages/torch/onnx/symbolic_helper.py:1513: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'instance_norm' is set to train=True. Exporting with train=True.
  warnings.warn(


In [7]:
model = SelfAttention(512)
dummy = torch.randn(1, 512, 32, 32)

traced = torch.jit.trace(model, dummy)
traced.save("selfattention.pt")

![attn](./graph/selfattention.pt.svg)

🔍 この Self-Attention は何？

これは Non-Local Attention（Self-Attention） の典型実装で、
特徴マップ内の すべての位置同士の関連性を計算して、
「どこがどこと関係しているか」を学習します。

線画生成だと：

遠く離れた線の方向性

同じ構造の反復

長い輪郭線のつながり

などを補完できるので U-Net によく使われます。

Self-Attention（Non-Local Block）を U-Net に追加した場合、実質的に “新しいハイパーパラメータはほぼ生まれません。”
ただし、設置位置・数・in_dim（チャネル）・学習率への影響 という “準ハイパーパラメータ” は存在します。 

★ ユーザーが調整する必要がある（準ハイパーパラメータ）

どの層に入れるか（最重要）

何個入れるか

学習率を変えるか（場合により）

in_dim（チャネル数）の合う層を選ぶ必要

★ ユーザーが調整しなくてよい（ハイパーパラメータでない）

γ

attention の温度

head 数

dropout

kernel size

scaling factor

これらはすべて自動で機能するので触らない。

In [6]:
model = AttnResUNet()
dummy = torch.randn(1,1,256,256)

out = model(dummy)
dot = make_dot(out, params=dict(model.named_parameters()))
dot.render("attnres_graph", format="png")

'attnres_graph.png'

![attnres](./graph/attnres_graph.png)